In [ ]:
import torch
import torch.nn as nn

# Training a Neural Network

## Softmax 층

$$y_i = \text{Softmax}(z_i) = \dfrac{\exp(z_i)}{\sum_j^k \exp(z_j)}$$

`torch.softmax` 를 사용하면 함수처럼 사용할 수 있고, `nn.Softmax()`를 사용해서 객체 생성 후 하나의 층 처럼 사용할 수도 있다. 다만 Softmax 할 차원을 지정해줘야한다.

* PyTorch Docs: [softmax](https://pytorch.org/docs/stable/nn.html#softmax)

In [ ]:
x = torch.FloatTensor([3, 1, 5, 9])
prob = torch.softmax(x, dim=0)
print(f"{prob.numpy().round(3)}")

[0.002 0.    0.018 0.979]


In [ ]:
softmax_layer = nn.Softmax(dim=1)
x = torch.FloatTensor([[3, 1, 5, 9], 
                       [4, 6, 5, 3]])
print(x.size())
prob = softmax_layer(x)
print(f"{prob.numpy().round(3)}\n")
print(f"dim=1 으로 합을 하면 1이 된다: {prob.sum(1)}")

torch.Size([2, 4])
[[0.002 0.    0.018 0.979]
 [0.087 0.644 0.237 0.032]]

dim=1 으로 합을 하면 1이 된다: tensor([1.0000, 1.0000])


보통 softmax의 0에 가까운 아주 작은 수라서 log를 취해서 `torch.log_softmax`를 사용한다(혹은 `nn.LogSoftmax`).

In [ ]:
torch.log_softmax(x, dim=1)

tensor([[-6.0209, -8.0209, -4.0209, -0.0209],
        [-2.4402, -0.4402, -1.4402, -3.4402]])

## Loss Function

XOR 문제는 0과 1 두 가지 클래스를 분류하는 문제다. Cross-Entropy 를 사용할 수 있다. 

따라서 네트워크의 마지막 층을 2개로 출력하고 `Softmax` 층을 넣어야 한다. 다만 PyTorch의 Cross Entropy Loss(`nn.CrossEntropyLoss`)에는 `LogSoftmax`가 포함되어 있어서 넣지 않아도 된다.

In [ ]:
torch.manual_seed(70)

class XOR(nn.Module):
    """XOR Network"""
    def __init__(self):
        super(XOR, self).__init__()
        # 층을 구성
        self.layers = nn.Sequential(
            nn.Linear(2,4),  # in_features, out_features
            nn.Sigmoid(),
            nn.Linear(4,2),
            nn.Sigmoid(),
            nn.Linear(2,2),
        )

    def forward(self, x):
        # forward propagation 수행
        o = self.layers(x)
        return o
    
    def predict(self, x):
        o = self.forward(x)
        y = torch.softmax(o, dim=1)
        return y

In [ ]:
# 입력텐서 타겟 텐서 생성    
x = torch.Tensor([[0, 1]])
t = torch.LongTensor([0])

# 커스텀 모듈 호출
model = XOR()

# 손실함수
loss_function = nn.CrossEntropyLoss()

# 순방향전파
y = model(x)

# 손실값 계산
loss = loss_function(y,t)

print(f"loss value: {loss.item()}")

loss value: 0.33104363083839417


## nn.AutoGrad

PyTorch의 AutoGrad는 Tensor의 미분 자동화를 돕는 패키지다. 각 텐서에는 `requires_grad`라는 속성이 있어서 미분이 필요한 텐서인지 아닌지 확인할 수 있다. 또한 `requires_grad_` 함수를 호출하면 해당 텐서는 미분이 필요한 텐서가 되며, 역전파시 미분을 계산하게 된다.

* 함수뒤에 "\_" 표시는 in-place operations 으로써 실행하면 새로운 메모리에 할당하지 않고, 메모리를 차지하고 있는 텐서에 덮어쓰는 형식이다. PyTorch에서는 사용을 권장하고 있지 않다. [관련 링크](https://pytorch.org/docs/stable/notes/autograd.html#in-place-operations-with-autograd)

In [ ]:
x = torch.FloatTensor([10])

print(f"require gradient? {x.requires_grad}")
print(x)
print()

# requires_grad
x.requires_grad_(True)
# '_' x의 특성을 바꾼다.
print(f"require gradient? {x.requires_grad}")
print(x)

require gradient? False
tensor([10.])

require gradient? True
tensor([10.], requires_grad=True)


<img src="https://drive.google.com/uc?id=1SPGm636Na_VrRTHkcBhMOQGMq0CaYAtg" width="640px" >

예제로 계산 그래프를 그려본다.

$$\begin{aligned}
c(a, b) &= a + b\\
d(b) &= 2\times b + 1\\
e(c, d) &= c\times d 
\end{aligned} \\ \ \\ \text{where } a=2, b=3$$

연산 경로에 미분이 필요한 텐서가 들어가면 자동으로 `requires_grad=True`가 된고, 연산이 진행된 텐서는 `grad_fn` 역전파 함수를 내포하고 있다.

In [ ]:
a = torch.FloatTensor([2]).requires_grad_()
b = torch.FloatTensor([3])
c = a + b
d = 2 * b + 1
e = c * d

print(f"require gradient?")
for t, name in zip([a, b, c, d, e], ["a", "b", "c", "d", "e"]):
    print(f"  - {name}(={t.item()}): {t.requires_grad} \t/ grad_fn: \t {t.grad_fn}")

require gradient?
  - a(=2.0): True 	/ grad_fn: 	 None
  - b(=3.0): False 	/ grad_fn: 	 None
  - c(=5.0): True 	/ grad_fn: 	 <AddBackward0 object at 0x7f06a543cfa0>
  - d(=7.0): False 	/ grad_fn: 	 None
  - e(=35.0): True 	/ grad_fn: 	 <MulBackward0 object at 0x7f06a543cfa0>


미분을 구하려면 `backward` 함수에 경사를 전달하면 된다. 각 텐서에서 `.grad` 속성을 조회하면 미분값을 확인할 수 있다. 

In [ ]:
gradient = torch.FloatTensor([1.])
e.backward(gradient)

print(f"gradient")
for t, name in zip([a, b, c, d, e], ["a", "b", "c", "d", "e"]):
    print(f"  - {name}: {t.grad}")

gradient
  - a: tensor([7.])
  - b: None
  - c: None
  - d: None
  - e: None


<ipython-input-17-c806ec9d3b09>:6: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at aten/src/ATen/core/TensorBody.h:486.)
  print(f"  - {name}: {t.grad}")


## torch.optim

PyTorch의 최적화 관련된 것은 모두 optim 패키지에 있다. `model.parameters()`는 모델 안에 내포되있는 모든 파라미터를 `generator` 객체를 생성한다. 이를 옵티마이저에게 전달하여 업데이트할 매개변수를 등록한다. 

In [ ]:
import torch.optim as optim

# 입력텐서 타겟 텐서 생성    
x = torch.Tensor([[0, 1]])
t = torch.LongTensor([0])

# 커스텀 모듈 호출
model = XOR()

# 손실함수
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1) # 최적의 optimzier를 찾아가는 방법

# 순방향전파
y = model(x)

# 손실값 계산
loss = loss_function(y, t)

# 역전파
loss.backward()

In [ ]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.4407,  0.6366],
         [-0.3092, -0.1563],
         [-0.1584, -0.4085],
         [ 0.4569, -0.1494]], requires_grad=True),
 Parameter containing:
 tensor([-0.4993, -0.5181, -0.3645, -0.1876], requires_grad=True),
 Parameter containing:
 tensor([[ 0.1086,  0.3610, -0.3573,  0.1406],
         [-0.1545, -0.4260, -0.1394,  0.2492]], requires_grad=True),
 Parameter containing:
 tensor([0.0942, 0.2225], requires_grad=True),
 Parameter containing:
 tensor([[-0.3497,  0.0584],
         [ 0.3193, -0.2502]], requires_grad=True),
 Parameter containing:
 tensor([ 0.3200, -0.3378], requires_grad=True)]

옵티마이저의 `.step()` 함수를 호출하면 옵티마이저가 해당 매개변수를 업데이트 해준다.

In [ ]:
print("첫번째 Linear Layer Weight: ")
print(model.layers[0].weight)
print()
print("첫번째 Linear Layer Weight의 Gradient: ")
print(model.layers[0].weight.grad)
print()

optimizer.step()

print("첫번째 Linear Layer Weight의 Gradient: ")
print(model.layers[0].weight)

첫번째 Linear Layer Weight: 
Parameter containing:
tensor([[-0.4407,  0.6366],
        [-0.3092, -0.1563],
        [-0.1584, -0.4085],
        [ 0.4569, -0.1494]], requires_grad=True)

첫번째 Linear Layer Weight의 Gradient: 
tensor([[ 0.0000,  0.0029],
        [ 0.0000,  0.0081],
        [-0.0000, -0.0041],
        [ 0.0000,  0.0004]])

첫번째 Linear Layer Weight의 Gradient: 
Parameter containing:
tensor([[-0.4407,  0.6363],
        [-0.3092, -0.1571],
        [-0.1584, -0.4081],
        [ 0.4569, -0.1494]], requires_grad=True)


경사하강법 $$ W^{(1)}_{new} = W^{(1)}_{old} - \alpha \dfrac{\partial L}{\partial W^{(1)}}_{old}$$

In [ ]:
print(f" w_22 의 파라미터 업데이트: w ={0.3146 - 0.1 * (0.0074): .4f}")

 w_22 의 파라미터 업데이트: w = 0.3139


## XOR 문제 학습하기

In [ ]:
torch.manual_seed(70)

device = "cuda" if torch.cuda.is_available() else "cpu"  # gpu 사용 여부
n_step = 10000  # 총 학습 스텝

# Data 세트 만들기
inputs = torch.FloatTensor([[0, 0], [1, 0], [0, 1], [1, 1]])
targets = torch.LongTensor([0, 1, 1, 0])

# 모델 생성: gpu를 사용하려면 모델에도 device를 전달해준다.
model = XOR().to(device)
# 손실함수 정의
loss_function = nn.CrossEntropyLoss()
# 옵티마이저 정의
optimizer = optim.SGD(model.parameters(), lr=0.7)

# GPU를 사용하려면 입력 텐서에도 device를 전달해준다.
inputs, targets = inputs.to(device), targets.to(device)

best_loss = 999
# n_step 동안 학습을 진행한다.
for step in range(n_step):
    # -- 훈련단계 --
    train_loss = 0
    
    # 매개변수 텐서의 grad 정보를 0으로 만든다. model.zero_grad() 로도 가능하다.
    optimizer.zero_grad()
    
    # 순방향전파(Forward Propagation)
    outputs = model(inputs)
    
    # Loss 계산
    loss = loss_function(outputs, targets)
    
    # 역방향전파(Back Propagation)
    loss.backward()
    
    # 옵티마이저로 매개변수 업데이트
    optimizer.step()
    
    # 훈련단계 손실값 기록(모든 데이터에 손실값의 평균을 합친다.)
    train_loss += loss.item()
    if train_loss < best_loss:
        best_loss = train_loss
        torch.save(model.state_dict(), "./xor.pt")
    if step % 1000 == 0:
        print(f"[{step+1}] Loss: {train_loss:.4f}")

[1] Loss: 0.8012
[1001] Loss: 0.6842
[2001] Loss: 0.0035
[3001] Loss: 0.0012
[4001] Loss: 0.0007
[5001] Loss: 0.0005
[6001] Loss: 0.0004
[7001] Loss: 0.0003
[8001] Loss: 0.0003
[9001] Loss: 0.0002


In [ ]:
model.state_dict()

OrderedDict([('layers.0.weight',
              tensor([[-0.7095, -0.1966],
                      [ 1.7193, -2.0518],
                      [ 5.7520, -5.7997],
                      [ 3.9542, -4.0161]], device='cuda:0')),
             ('layers.0.bias',
              tensor([-0.6352, -0.9947,  3.0022, -2.2557], device='cuda:0')),
             ('layers.2.weight',
              tensor([[-0.4618,  2.1358, -5.5186,  4.5091],
                      [-0.2662, -1.8281,  4.6862, -4.1580]], device='cuda:0')),
             ('layers.2.bias', tensor([ 2.2529, -1.6060], device='cuda:0')),
             ('layers.4.weight',
              tensor([[-6.4570,  5.3559],
                      [ 5.7851, -5.4662]], device='cuda:0')),
             ('layers.4.bias', tensor([-0.2816, -0.5573], device='cuda:0'))])

## 모델 사용하기

In [ ]:
inputs

tensor([[0., 0.],
        [1., 0.],
        [0., 1.],
        [1., 1.]], device='cuda:0')

In [ ]:
# 모델 새로 정의
model = XOR()
# 모델 불러오기
model.load_state_dict(torch.load("./xor.pt", map_location="cuda"))

probs = model.predict(inputs.cpu())
print(probs)
predicts = probs.argmax(1)
print(predicts)
for prob, pred in zip(probs, predicts):
    print(f"prob: {prob.data}\t predict {pred}")

tensor([[9.9977e-01, 2.2552e-04],
        [1.6342e-04, 9.9984e-01],
        [2.2162e-04, 9.9978e-01],
        [9.9983e-01, 1.7327e-04]], grad_fn=<SoftmaxBackward0>)
tensor([0, 1, 1, 0])
prob: tensor([9.9977e-01, 2.2552e-04])	 predict 0
prob: tensor([1.6342e-04, 9.9984e-01])	 predict 1
prob: tensor([2.2162e-04, 9.9978e-01])	 predict 1
prob: tensor([9.9983e-01, 1.7327e-04])	 predict 0
